# project IV

撰写者：12112510李涵

指导老师：于仕琪

# 第一部分：项目简述

使用C++，为卷积神经网络的data blobs设计一个类，它可以作为卷积层的输入，输出和卷积核。这个类应该包含data blob的数据和相关信息（如行数，列数，通道数e.t.c）。这个类应该能支持不同的数据类型（整数浮点数之类的）。注意内存泄漏问题，如果data blob被分配给另外一个，不要使用深复制。对操作符进行重载。可以参考cv::Mat来实现。内存管理是重点！！！

为简便起见，本次project的所有数据默认只有三维（3 dimensions），即长度、宽度和通道数。

# 第二部分：基础内容阐述

## 总体设计

本次项目主要有以下文件：  test.cpp, blob.hpp

其中头文件定义了两个模板类的属性和方法（和具体方法的实现）。test.cpp则是对程序的测试。

## 参数定义：

1. rows_：行数
2. cols_：列数
3. channels_：通道数
4. steps_：每一维元素的大小
5. data_：指向存储的数据的首地址值
6. memory_pool_：静态内存池变量，所有Blob对象公用。

## 构造器：

**Blob()：**

所有参数默认为$0$，不会为`data`分配内存空间。

**Blob(size_t rows, size_t cols, size_t channels)：**

会为`data`分配一块大小为$rows * cols * channels$的内存区域，用`make_shared`来创建一个智能指针指向这块区域。

**Blob(const Blob &other)：**

对目标对象进行深拷贝的构造函数。

## 拷贝：

**clone():**

实现了深拷贝。即，在拷贝对象被修改或释放的情况下，自身也不会受到任何影响。

**=：（等于号的重载）**

实现了浅拷贝。自身和拷贝对象指向同一块内存区域。当一个对象被释放后，会进行一次检测。如果这块内存区域仍有其他对象指向，则不会进行释放。在cv::Mat中，是通过`refCount`属性来实现的。`refCount`的内存空间紧随`data`之后，记录了指向`data`的对象的个数。当一个对象被释放后，`refCount`会减一。在C++的近期版本中，可以使用智能指针`shared_ptr`来简单粗暴地直接实现这一功能。

## 访问：

### 1. at访问：

**定义：**

at访问是一种基于多维索引的访问方式。通过给定的行、列和通道（对于图像而言）索引，可以直接访问到存储在多维数组或矩阵中的元素。这种访问方式通常使用一个索引计算公式将多维索引转换为一维数组中的索引。

**优势：**

1. 直观：使用at访问可以直接通过多维索引访问到数据，而无需关心底层的一维存储方式。
2. 容错：at访问通常包含边界检查，可以确保访问的索引在有效范围内，从而避免越界访问。

**相关公式：**

$index = rows\_ * cols\_ * channel + rows\_ * col + row$

注意：其实有两种常见的方式来存储三维数据（存到一维数组中）：

1. 先通道后空间（channel-first）：在这种情况下，每个通道的所有元素都是连续存储的。所以，对于给定的行、列和通道，索引将是$(channel * rows * cols) + (row * cols) + col$。
2. 先空间后通道（space-first）：在这种情况下，每个空间位置的所有通道都是连续存储的。所以，对于给定的行、列和通道，索引将是$(row * cols * channels) + (col * channels) + channel$。

在这里选用先通道后空间。

<font color=FF0000>但通常来说并不会使用at访问，opencv官方更推荐使用ptr访问，原因详见后续</font>

### 2. 迭代器访问：

**定义：**

迭代器是一个指针类对象，用于在容器（如数组、向量、链表等）中遍历元素。迭代器可以通过递增或递减操作符（++或--）来移动到下一个或上一个元素。

**优势：**

1. 通用：迭代器可以用于访问各种容器中的元素，而不仅仅局限于多维数组或矩阵。
2. 高效：迭代器访问通常不涉及边界检查，因此在某些情况下（如连续遍历整个容器时）可能比at访问更快。
3. 灵活：迭代器可以方便地在容器中移动，实现对元素的插入、删除等操作。

**相关解释：**

1. begin(): 返回一个指向容器（std::vector<T>）第一个元素的迭代器。对于非常量对象，返回的是 std::vector<T>::iterator 类型的迭代器。
2. cbegin(): 类似于 begin()，但返回的是 std::vector<T>::const_iterator 类型的迭代器，用于常量对象的访问。
3. end(): 返回一个指向容器（std::vector<T>）最后一个元素之后的迭代器。对于非常量对象，返回的是 std::vector<T>::iterator 类型的迭代器。
4. cend(): 类似于 end()，但返回的是 std::vector<T>::const_iterator 类型的迭代器，用于常量对象的访问。

### 3. ptr访问：

**定义：**

当我们使用一个多维数组或矩阵来存储数据时，实际上这些数据在内存中是连续存储的。为了访问特定位置的数据，我们需要计算其在内存中的偏移量。ptr方法通过计算每个维度的步长来实现这一点。

**优势：**

1. 速度：ptr访问方法使用指针直接访问内存，因此访问速度非常快。相比于其他访问方法（如at方法），它减少了一些额外的计算和函数调用开销，从而提高了性能。

2. 灵活性：ptr方法允许访问内存中连续的数据块，这意味着我们可以更轻松地处理大型数据集，例如图像和多维数组。此外，ptr方法也可以与其他指针操作（如指针运算和类型转换）结合使用，从而提供更高的灵活性。

3. 与底层硬件更接近：ptr访问直接操作内存地址，更接近底层硬件。这使得在某些情况下，我们可以更好地利用硬件优势，例如缓存局部性。

**相关公式：**

在计算每个维度的步长时，我们需要知道每个维度的大小。例如，在一个3维Blob（行、列、通道）中，我们需要知道行数、列数和通道数。然后，我们可以计算每个维度的步长：

1. 行步长：$rows\_step = cols * channels * sizeof(T)$
2. 列步长：$cols\_step = channels * sizeof(T)$
3. 通道步长：$channels\_step = sizeof(T)$

`sizeof(T)`表示数据类型T的大小（以字节为单位）。对于浮点数（`float`），它通常是4字节；对于双精度浮点数（`double`），它通常是8字节。

然后，为了访问Blob中特定位置的数据（如第i行、第j列、第k通道），我们可以使用以下公式：

$ \&data(i, j, k) = data\_ptr + i * rows\_step + j * cols\_step + k * channels\_step $

`data_ptr`是指向Blob数据开始位置的指针。通过将这个公式与指针运算结合起来，我们可以高效地访问任意位置的数据。并且因为返回的是指针，它就有更高的灵活性，使得程序有更多的选择性。

PS：好像是说cv::mat里面的ptr不提供边界检查。。？但我觉得这样好危险。。还是加上了）

### 一些问题：

**Q：为什么使用一维数组来存储数据（而不是多维数组）？**

1. 在多维数组中，每个元素占用的内存空间不一定是相邻的（比如动态分配内存时，操作系统可能在内存中的不同位置找到空闲空间）。而在一维数组中，所有元素都是相邻的，这有助于提高内存访问的效率。
2. 在多维数组中，内存访问需要进行更多的计算，例如计算每个元素在内存中的位置，而在一维数组中，这些计算可以被避免。

**Q：使用`size_t`而不是`int`的原因？**

1. 在矩阵中使用`size_t`类型通常更安全和合适，因为`size_t`是一个无符号整数类型，它能够存储非负数，而矩阵的大小通常是非负数，因此`size_t`可以更好地表示这种情况。

**Q：为什么参数后面都要加一个下划线？**

1. 下划线被用来区分成员变量和局部变量，或者区分公有成员和私有成员。本次项目的参数名后面加下划线，就是为了防止与内置类型或标准库中的名称冲突，并且与类中的成员变量或函数区分开来。

**Q：为什么我不能在hpp定义类和参数和方法，然后在cpp文件里面include头文件，再实现这些方法呢？还是说模板类不允许我这么做？**

1. （来自gpt）对于模板类（template classes）和模板函数（template functions），情况就有所不同了。模板在C++中是一种特殊的编程机制，它们需要在声明的地方就提供完整的定义，因为模板的实例化（即为模板提供特定类型参数）是在编译时进行的。编译器必须在模板的声明点就看到完整的定义，才能生成相应类型的模板实例。如果你尝试在 .cpp 文件中实现模板类或模板函数，编译器可能无法找到这些实现，因为编译器在编译 .cpp 文件时，并不知道模板需要实例化为哪些类型。因此，对于模板类和模板函数，我们通常在头文件中同时提供声明和定义，或者在头文件中提供声明，然后在另一个头文件中提供定义，并在声明的头文件中包含定义的头文件。这样，当其他代码包含这个头文件时，它们就能看到模板的完整定义，从而可以实例化模板。

**PS：我来回改来改去好多遍最后告诉我不能这样做，我真的是吐了，我改了五六个小时了啊啊啊啊**

# 第三部分：各类优化

大多优化都是参照cv::mat进行的。~（如有雷同，不是巧合）~

### 1. 延迟内存分配：

**定义：**

贯彻了懒汉式的思维。即不会在构造器内立即为`data_`分配内存空间，只有当`data_`中的内存空间真正被访问和调用时，才去分配。

**实际操作：**

~在访问内存空间（如at，ptr访问等方法）等操作时，先检查`data_`是否为`null`，如果是的话，调用`allocate_if_needed()`方法。需要注意的是，这样可能会降低运行效率，但这样能在一定程度上节约内存空间（只能说是牺牲性能换取存储了。。）~

但是这样做的话，会违背一些const方法的定义。所以为了避免这样（同时也是为了提高效率），我选择将这个判断交给用户（假定用户非常严谨）！将这个方法改为了public，由用户在进行操作前判断。

### 2. 高效的内存复用：

通过设计内存池（Memory pool）类来实现。

**定义：**

这是一种内存管理策略，其主要目的是减少动态内存分配和释放的次数，从而提高程序运行的性能。内存池的原理是预先分配一定量的内存块，并在需要的时候从内存池中获取内存，不需要时归还给内存池。

**作用：**

1. 提高内存分配和释放的速度：通过减少动态内存分配和释放的次数，降低内存分配和释放的开销，从而提高程序的运行速度。
2. 减少内存碎片：内存池通过预先分配内存块，可以有效地减少内存碎片的产生，提高内存的利用率。
3. 方便内存管理：使用内存池可以方便地统一管理内存资源，便于检测内存泄漏和分析内存使用情况。

**实际操作：**

首先对内存池类做一个简单的介绍。

1. pool_：元素类型为`std::pair<std::shared_ptr<std::vector<T>>, bool>`，用于存储内存块以及它们的分配状态。
2. first：表示内存块。
3. second：表示内存块的分配状态。

`request_memory()`：首先遍历`pool_`，查找一个可用的内存块。如果找到一个可用的内存块（即`second`为true），我们将其状态更改为已分配（false），并将`std::shared_ptr<std::vector<T>>`返回给调用者。如果在`pool_`中找不到可用的内存块，我们将创建一个新的内存块，并将其添加到`pool_`中。这里使用`std::vector`的`push_back()`函数将新创建的内存块添加到`pool_`中。`push_back()`函数将元素添加到`std::vector`的末尾。

`release_memory()`：遍历`pool_`，查找与传入的`std::shared_ptr<std::vector<T>>`相匹配的内存块。找到匹配的内存块后，我们将其状态更改为可用(true)。

接下来是将其贯彻到`Blob`类的代码中。

**属性：** 添加`static MemoryPool<T> memory_pool_;`，使得所有对象共用这一个内存池。同时为了初始化它，需要在头文件的最下方，添加
```C++
template<typename T, typename Enable>
MemoryPool<T> Blob<T, Enable>::memory_pool_;
```
来确保其在程序刚运行时便初始化完毕。

**构造函数：** 调用`request_memory()`方法，为`data_`分配指定大小的内存空间。

**析构函数：** ~在`data_`不为空，且这块`data_`的指向数量仅为1时（使用std::shared_ptr的use_count()成员函数来获取data_的引用计数），调用`release_memory()`释放这块内存空间。~ 在后面内存对齐模块，会有新的释放内存的方案。

**PS：**

如果没有可用的内存块，就去创建一个新的内存块的话，可能会导致内存的过度使用。所以我设置了一个限度（`static constexpr size_t max_memory_block_size`)。申请小于这个的内存，就使用内存池，否则就直接申请内存，绕过内存池。

### 3. 避免不必要的内存分配＆释放：

**定义：**

在一些运算方法中（比如矩阵转置和翻转），如果不要求保留原始数据，并且大小规模前后差异微乎其微的话，便可应用原地操作（in-place operation），不创建新的对象，而是直接对原始数据进行修改。

另外，为了保证便用性，程序同时提供两种方法（修改&不修改原始数据）。

**实际操作：**

（一些样例）见`Blob<T> transposeWithReturn() const`&`void transpose()`以及`Blob<T> flipWithReturn() const`&`void flip()`。

### 4. 子矩阵视图

**定义：**

这是一种数据结构，它允许你访问原始矩阵的一部分，而无需复制数据。实际用途较为广泛，在此泛泛列举几个：

1. 区域感兴趣（ROI，Region of Interest）：在图像处理过程中，你可能只对图像的某个特定区域感兴趣。使用子矩阵，你可以专注于这个区域，而不必处理整个图像。这可以提高处理速度并减少资源需求。
2. 图像分割（Image segmentation）：在计算机视觉中，将图像划分为多个区域（例如前景和背景）是一个常见任务。通过创建每个区域的子矩阵视图，你可以更轻松地处理每个区域。
3. 特征提取（Feature extraction）：在计算机视觉任务中，特征提取是关键步骤。例如，在人脸识别中，你可能需要提取眼睛、鼻子和嘴巴等局部特征。使用子矩阵可以帮助你更高效地处理这些局部区域。

**实际操作：**

通过`T* subBlob_data = data_ + (start_row * cols_ + start_col);`创建从指定区域开始的新指针，并通过软拷贝的构造函数初始化一个新的`Blob`（详见`Blob<T> subBlob(int start_row, int start_col, int end_row, int end_col)`)

### 5.move方法

**定义：**

`std::move`是C++11标准库中的一个实用函数，它可以将一个左值转换为右值引用。这有助于在某些情况下实现资源的转移，而不是复制，从而提高程序的性能。一些使用场景诸如：

1. 当一个对象即将销毁时，将其资源移动到新对象中，而不是复制，可以提高性能。
2. 在处理临时对象（右值）时，可以直接移动资源，而不是复制。
3. 当编写容器或其他管理资源的类时，可以实现移动构造函数和移动赋值操作符，以优化复制操作。

**作用：**

1. 提高性能：通过移动资源而不是复制，可以避免不必要的内存分配和释放操作，从而提高程序的性能。
2. 简化代码：在某些情况下，使用移动语义可以简化代码，因为你不再需要编写复制构造函数和赋值操作符。
3. 支持不可复制的类型：有些类型是不可复制的（如 std::unique_ptr），而通过移动语义，我们可以在这些类型之间传递资源。

**具体实现：**

新开了一个`Blob moveFrom(Blob&& other)`方法，将`other`的数据转移给`this`。具体就是多加了一行move data。

### PS：避免使用std::vector作为内部数据结构。。！

由于`std::vector`会自动管理其内存，并且可能在需要时重新分配内存，这可能导致额外的内存开销。其实可以考虑使用裸指针（例如`T*`）作为`Blob`内部数据的底层存储。这将允许我更精确地控制内存分配和回收。（但是，这样做的缺点是需要更多的手动内存管理，然后真的好麻烦，所以就没写了。。）

# 第四部分：健壮性&&便用性

本节会讲述一下本项目为了提高程序的健壮性（不那么容易崩掉）和便用性（不那么难用）而采用的措施。

### 1. 检查操作合理性。

**实际操作：**

无论进行任何操作，都要检查数据的合法性。比如作运算时，用`check(other)`检查两个矩阵的大小是否匹配，`data_`是否为空，如果是的话，直接抛出RTE（这种操作太危险了，必须立刻中断！）（这也可能会降低运行效率。。但毕竟安全第一啦）

### 2. 运算符的重载

**实际操作：**

在运算前，会先检查两个矩阵的规模是否符合。然后创建两个指针，指向每个`data_`的首地址值。因为`data_`采用一维初始化，所以其储存的元素必然（应该把）连续。故而直接通过`++ptr`的方式遍历矩阵元素。这样能减少计算地址所花费的时间。

另外，注意到，对于乘除法操作，除了两个矩阵相乘，还有可能是一个矩阵与一个标量相乘除，所以也特地为了这个，而多重载了两次运算符。（详见`Blob operator*(const Blob &other) const`和`Blob operator*(const T &scalar) const`。另外在标量除法时，会检查标量是否为0，如果是的话直接报错！！）

### 3. 错误处理和异常

这里参考了老师上课提到的，输出出错的位置（代码的行数），并且会打印出错的数据和正确的数据，以此来给予用户更详尽的信息。

**实际操作：**

一个例子如下：
```C++
        if(row > rows_ || col > cols_ || channel > channels_){
            std::ostringstream os;
            os << "Error at " << __FILE__ << ":" << __LINE__ << "\n";
            os << "Index out of range: row(" << row << "), col(" << col << "), channel(" << channel << ")\n";
            os << "Valid range: row(0-" << rows_ << "), col(0-" << cols_ << "), channel(0-" << channels_ << ")";
            throw std::runtime_error(os.str());
        }
```

PS：因为\__FILE__和\__LINE__只能给出抛出异常的位置，如果异常是在一个函数中抛出的，而这个函数在多个地方被调用，那么这些信息可能并不能准确地反映出问题的来源。对于这种情况，可能需要一些更复杂的调试技术，如栈回溯（stack traceback）。所以诸如`check()`等检查方法便没有使用该功能了。。。

### 4. 类型检查

程序员需要考虑用户的一切可能，包括但不限于奇奇怪怪的输入）

如果类模板对于`T`类型没有任何限制，这可能会导致一些错误。例如，如果`T`是一个没有默认构造函数的类型，那么`std::vector<T>(size)`就会出错。所以我们需要添加一些静态断言，以确保 T 类型满足需求。

**实际操作：**

在两个类的上面，使用`std::enable_if`创建了一个只在`T`是指定类型之一时才有效的类型。如果`T`不是这些类型之一，那么`std::enable_if`没有`::type`成员，j就会导致编译错误。这样，如果尝试使用一个不支持的类型实例化两个类，编译器就会报错。

具体代码如下：
```C++
template<typename T, typename Enable = typename enable_if<
        is_same<T, char>::value ||
        is_same<T, int>::value ||
        is_same<T, long>::value ||
        is_same<T, unsigned>::value ||
        is_same<T, float>::value ||
        is_same<T, double>::value>::type>
```

# 第五部分：其他的一些之前的优化

### 1. 内存对齐

**定义：**

对齐内存是一种内存分配策略，即在内存地址上满足一定的整数倍约束。（好处包括性能优化，硬件要求，减少内存碎片等，在此不再赘述了）

**实际操作：**

先以`raw_memory`，用`std::aligned_alloc`来分配指定对齐要求的内存。然后将其转化为`shared_ptr`，并在已分配的内存上构造vector。

因为方法变化了，所以现在释放内存时，更改为先清空vector中的元素，并调用`release_memory()`，将内存归还给内存池。相应的，`release_memory()`现在接受的参数为`std::vector<T>* memory`。

# 第六部分：测试

接下来便是在`main()`中，实例化几个`Blob`对象，并通过各种各样的测试来验证代码的完备性。接下来分块解析：

* 对于b1, b2, b3，测试了软拷贝和硬拷贝是否正常工作。在修改b1后，可以看到b2的值发生了变动，而b3没有。
* 接下来测试了用迭代器和ptr来访问blob的字段，可以看到正常访问了。
* 接下来是两个乘法（向量和标量），可以看到结果正常，且乘的过程中并没有改变b1或b2的值。
* 接下来分别测试了翻转和裁剪等一系列功能，可以看到正常工作。
* b11测试了字段空间是否正常释放，将b11的字段指向b12，释放b12，可以看到b11的字段依然正常。
* 最后是测试报错，故意除以0，可以看到显示了报错提示和错误位置。

完整截图如下：
![FHW3P9N_I8TG3D_C48RMX8L.png](https://s2.loli.net/2023/05/13/CkyTqgL24OrG5du.png)

然后是在ARM平台作测试：

因为没能领到开发板。。只能自己在电脑上装ARM虚拟环境了。。

这里采用的是QEMU模拟器，详细安装流程参照[这里](https://zhuanlan.zhihu.com/p/479666128)

好吧我放弃了，根本装不了，不知道为什么，又花了毫无意义的六个小时。。就这样吧。。（没有热情了，筋疲力尽了。。）

# 第七部分：总结

emm感觉好像没上次那么困难（恶心）？可能四十多个小时就整完了。。（debug花了好久时间。。）

总的来说，本次project学到了：
* 模板编程：在C++中，模板是一种强大的工具，可以用于创建在类型上具有一定灵活性的代码。通过模板复用，可以大大简化代码和提高美观度和可读性。
* 内存管理：对诸如自定义的内存池等东东的实现，我更加理解了内存分配和释放的细节，以及如何优化内存使用以提高性能。
* 异常处理：通过抛出和捕获异常学习了如何处理可能在运行时发生的错误。我还学到了如何使用异常来表示不同的错误类型，并提供有关错误原因的有用信息。
* 类型检查：知道了使用C++模板元编程的特性，对模板参数进行类型检查，从而限制了函数或类模板的使用范围。
* 高效的数据访问：通过使用如ptr函数来访问数据，成功学习了如何提高数据访问的效率，特别是在处理大规模数据时。